In [2]:
import requests
import pandas as pd

# Define variables for the URL parameters and filename
tz_date = 'Europe%2FBerlin'
beginning_date = '2023-02-27'
end_date = '2023-07-22'
latitude = 52.425394
longitude = 9.867977
units = 'dwd'
excel_filename = 'weather_data.xlsx'

# Construct the URL using the defined variables
url = f'https://api.brightsky.dev/weather?date={beginning_date}&last_date={end_date}&lat={latitude}&lon={longitude}&tz={tz_date}&units={units}'
headers = {'Accept': 'application/json'}

# Make the GET request
response = requests.get(url, headers=headers)

In [3]:


# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    weather_data = data.get('weather', [])
    df = pd.DataFrame(weather_data)

    # Keep only specified columns
    columns_to_keep = ['timestamp', 'temperature', 'precipitation']
    df = df[columns_to_keep]

    # Convert timestamp to datetime and filter by time range
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    df = df[(df['timestamp'].dt.hour >= 5) & (df['timestamp'].dt.hour <= 14)]

    # Group by date and aggregate data
    df['date'] = df['timestamp'].dt.strftime('%d.%m.%Y')
    grouped = df.groupby('date').agg({
        'temperature': lambda x: ','.join(map(str, x)),
        'precipitation': lambda x: ','.join(map(str, x))
    }).reset_index()

    # Add column for temperature under 5 degrees
    df['temperature_under_5'] = df['temperature'].apply(lambda x: 'Ja' if any(float(temp) < 5 for temp in str(x).split(',')) else 'Nein')

    # Save to Excel
    grouped.to_excel(excel_filename, index=False)
    print(f"Weather data saved to {excel_filename}")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Weather data saved to weather_data.xlsx


In [4]:
df

,timestamp,temperature,precipitation,date,temperature_under_5
6,2023-02-27 05:00:00+00:00,-4.1,0.0,27.02.2023,Ja
7,2023-02-27 06:00:00+00:00,-3.3,0.0,27.02.2023,Ja
8,2023-02-27 07:00:00+00:00,-3.3,0.0,27.02.2023,Ja
9,2023-02-27 08:00:00+00:00,-1.9,0.0,27.02.2023,Ja
10,2023-02-27 09:00:00+00:00,0.6,0.0,27.02.2023,Ja
...,...,...,...,...,...
3467,2023-07-21 10:00:00+00:00,20.2,0.0,21.07.2023,Nein
3468,2023-07-21 11:00:00+00:00,20.2,0.0,21.07.2023,Nein
3469,2023-07-21 12:00:00+00:00,19.5,0.0,21.07.2023,Nein
3470,2023-07-21 13:00:00+00:00,19.5,0.0,21.07.2023,Nein
